In [ ]:
# Cell 1: Setup
%load_ext autoreload
%autoreload 2

import pandas as pd
from fund_lens_etl.clients.fec import FECAPIClient
from fund_lens_etl.extractors.fec import (
    FECScheduleAExtractor,
    FECCommitteeExtractor,
    FECCandidateExtractor,
)
from fund_lens_models.enums import USState

# Test shared client
print("Testing FEC API Client...")
client = FECAPIClient()
print(f"✓ Client initialized with rate limiter")
print(f"  Rate limiter stats: {client.get_rate_limiter_stats()}")

In [ ]:
# Cell 2: Test Committee Extractor - Single Committee
print("="*60)
print("TEST 1: Extract Single Committee by ID")
print("="*60)

committee_extractor = FECCommitteeExtractor(api_client=client)

# Extract the committee from your test data
df = committee_extractor.extract(committee_ids=["C00840017"])

print(f"\nExtracted {len(df)} committee(s)")
if not df.empty:
    print("\nColumns:", df.columns.tolist())
    print("\nSample data:")
    print(df[["committee_id", "name", "committee_type", "state", "party"]].to_string())

In [ ]:
# Cell 3: Test Committee Extractor - By State
print("="*60)
print("TEST 2: Extract Committees by State (MD, cycle 2026)")
print("="*60)

df = committee_extractor.extract(
    state=USState.MD,
    cycle=2026,
)

print(f"\nExtracted {len(df)} committees")
if not df.empty:
    print("\nCommittee types:")
    print(df.groupby("committee_type").size())
    print("\nSample:")
    print(df.head()[["committee_id", "name", "committee_type", "party"]])

In [ ]:
# Cell 4: Test Candidate Extractor - By State
print("="*60)
print("TEST 3: Extract Candidates by State (MD, cycle 2026)")
print("="*60)

candidate_extractor = FECCandidateExtractor(api_client=client)

df = candidate_extractor.extract(
    state=USState.MD,
    cycle=2026,
)

print(f"\nExtracted {len(df)} candidates")
if not df.empty:
    print("\nOffice breakdown:")
    print(df.groupby("office").size())
    print("\nSample:")
    print(df.head()[["candidate_id", "name", "office", "district", "party"]])

In [ ]:
# Cell 5: Test Schedule A Extractor - Find Committee with Contributions
print("="*60)
print("TEST 4: Extract Schedule A - Single Page")
print("="*60)

schedule_a_extractor = FECScheduleAExtractor(api_client=client)

# Get candidate committees first
committees = schedule_a_extractor.get_candidate_committees(
    state=USState.MD,
    election_cycle=2026
)

print(f"Found {len(committees)} MD candidate committees")

# Try to find one with contributions
for committee in committees[:5]:  # Test first 5
    print(f"\nTrying: {committee['committee_name']} ({committee['committee_id']})")

    try:
        for df, metadata in schedule_a_extractor.extract_schedule_a_pages(
            committee_id=committee['committee_id'],
            election_cycle=2026,
            starting_page=1
        ):
            if len(df) > 0:
                print(f"✓ Found contributions!")
                print(f"  Page {metadata['page']}: {len(df)} records")
                print(f"  Total available: {metadata['total_count']} records")
                print("\n  Sample contribution:")
                print(df.head(1)[["contributor_name", "contributor_city", "contribution_receipt_amount", "contribution_receipt_date"]])
                break
            else:
                print(f"  No contributions yet")
                break
    except Exception as e:
        print(f"  Error: {e}")
        continue

In [ ]:
# Cell 6: Rate Limiter Stats
print("="*60)
print("Final Rate Limiter Stats")
print("="*60)
stats = client.get_rate_limiter_stats()
for key, value in stats.items():
    print(f"  {key}: {value}")